# Présentation du problème du sac à dos

Le problème du sac à dos, connu sous le nom de "Knapsack Problem" en anglais, est un problème d'optimisation combinatoire. Le problème tire son nom d'une situation analogue à celle de remplir un sac à dos avec le plus d'objets précieux possible, sans dépasser un poids maximum. Il a été discuté dans des écrits scientifiques dès 1897 par le mathématicien Mathews.

# Types de problème de sac à dos

- Sac à dos 0/1: Chaque objet ne peut être pris qu'une fois ou pas du tout.
- Sac à dos fractionnaire: Vous pouvez prendre des parties fractionnaires d'objets.
- Sac à dos multiple: Des variantes où plusieurs sacs à dos ou des contraintes supplémentaires sont introduites.

# Fonctionnement

Entrées:
- Un ensemble d'objets, chacun avec un poids et une valeur.
- Une limite de poids totale (capacité du sac à dos).

Objectif: 
- Sélectionner une combinaison d'objets qui maximise la valeur totale sans dépasser le poids total autorisé.

Approches de résolution:
- Programmation Dynamique: Utilisée pour le sac à dos 0/1, elle construit des solutions pour des capacités croissantes.
- Greedy (pour le sac à dos fractionnaire): Choisissez les objets avec le meilleur ratio valeur/poids jusqu'à ce que le sac soit plein.
- Méthodes exactes et heuristiques: Pour les variantes plus complexes ou pour des solutions plus rapides.



In [5]:
class Item:
    def __init__(self, name, weight, value):
        self.name = name
        self.weight = weight
        self.value = value

def remove_item(items, name):
    items = [item for item in items if item.name != name]
    return items

# Fonction qui calcule la valeur maximale du sac à dos
def knapsack(items, max_weight):
    n = len(items)
    K = [[0 for x in range(max_weight + 1)] for x in range(n + 1)]
    picked = [[[] for x in range(max_weight + 1)] for x in range(n + 1)]

    # Construction du tableau K[][] en bottom-up
    for i in range(n + 1):
        for w in range(max_weight + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif items[i-1].weight <= w:
                if items[i-1].value + K[i-1][w-items[i-1].weight] > K[i-1][w]:
                    K[i][w] = items[i-1].value + K[i-1][w-items[i-1].weight]
                    picked[i][w] = picked[i-1][w-items[i-1].weight] + [items[i-1]]
                else:
                    K[i][w] = K[i-1][w]
                    picked[i][w] = picked[i-1][w]
            else:
                K[i][w] = K[i-1][w]
                picked[i][w] = picked[i-1][w]

    # Retourne la valeur maximale et les items choisis
    max_val = K[n][max_weight]
    items_chosen = picked[n][max_weight]
    weight_sum = sum(item.weight for item in items_chosen)
    return max_val, items_chosen, weight_sum

items = [
    Item("lampe de poche", 68, 10),
    Item("boite d'allumettes", 50, 7),
    Item("boussole", 15, 6),
    Item("canif", 10, 5),
    Item("sac de couchage", 200, 20),
    Item("corde", 100, 15),
    Item("nourriture", 50, 17),
    Item("eau", 75, 18),
    Item("tente", 340, 25),
    Item("réchaud", 150, 24),
    Item("Pierre à feu", 75, 11),
]

max_weight = 250  

max_val, items_chosen, weight_sum = knapsack(items, max_weight)

print("> Mon sac à dos contient :")
for item in items_chosen:
    print(f"> {item.name}")
print(f"> son poids est de : {weight_sum}")
print(f"> sa valeur est égale à : {max_val}")


> Mon sac à dos contient :
> boussole
> canif
> corde
> nourriture
> eau
> son poids est de : 250
> sa valeur est égale à : 61


# Algorithme 1 : Recherche rapide d'une borne supérieure de la valeur optimale

L'algorithme implémenté ci-dessous, utilise une approche gloutonne (greedy) basée sur le rapport valeur/poids des objets. En sélectionnant d'abord les objets avec le meilleur rapport valeur/poids, l'algorithme maximise la valeur par unité de poids ajoutée au sac. Lorsque l'ajout d'un objet entier dépasse la capacité restante du sac, une fraction de cet objet est incluse pour remplir exactement le poids restant. Cette méthode assure d'obtenir la valeur maximale possible sans dépasser le poids limite, fournissant ainsi une borne supérieure de la valeur optimale. Mais cette borne n'est pas nécéssairement la solution optimals. 

In [11]:
class Item:
    def __init__(self, name, weight, value):
        self.name = name
        self.weight = weight
        self.value = value

def remove_item(items, name):
    items = [item for item in items if item.name != name]
    return items

def quicksort(items):
    if not items:
        return []
    pivot = items[0]
    lesser = quicksort([x for x in items[1:] if x.value / x.weight > pivot.value / pivot.weight])
    greater = quicksort([x for x in items[1:] if x.value / x.weight <= pivot.value / pivot.weight])
    return lesser + [pivot] + greater

def calculate_upper_bound(items, max_weight):
    sorted_items = quicksort(items)
    weight_accumulated = 0
    value_accumulated = 0
    for item in sorted_items:
        if weight_accumulated + item.weight <= max_weight:
            weight_accumulated += item.weight
            value_accumulated += item.value
        else:
            fraction = (max_weight - weight_accumulated) / item.weight
            weight_accumulated += item.weight * fraction
            value_accumulated += item.value * fraction
            break

    return value_accumulated

def calculate_upper_bound(items, max_weight):
    sorted_items = quicksort(items)
    weight_accumulated = 0
    value_accumulated = 0
    item_fractions = {}  # Stocker les fractions d'objets

    for item in sorted_items:
        if weight_accumulated + item.weight <= max_weight:
            weight_accumulated += item.weight
            value_accumulated += item.value
            item_fractions[item.name] = 1  # Objet entièrement pris
        else:
            fraction = (max_weight - weight_accumulated) / item.weight
            weight_accumulated += item.weight * fraction
            value_accumulated += item.value * fraction
            item_fractions[item.name] = fraction  # Fraction de l'objet pris
            break

    return value_accumulated, item_fractions

items = [
    Item("lampe de poche", 68, 10),
    Item("boite d'allumettes", 50, 7),
    Item("boussole", 15, 6),
    Item("canif", 10, 5),
    Item("sac de couchage", 200, 20),
    Item("corde", 100, 15),
    Item("nourriture", 50, 17),
    Item("eau", 75, 18),
    Item("tente", 340, 25),
    Item("réchaud", 150, 24),
    Item("Pierre à feu", 75, 11),
]

max_weight = 245

upper_bound_value, item_fractions = calculate_upper_bound(items, max_weight)
print("Valeur de la borne supérieure:", upper_bound_value)
print("Fractions des objets pris:")
for item, fraction in item_fractions.items():
    print(f"{item}: {fraction}")

Valeur de la borne supérieure: 61.2
Fractions des objets pris:
canif: 1
boussole: 1
nourriture: 1
eau: 1
réchaud: 0.6333333333333333


# Algorithme 3 : Calcul de la solution optimale exacte par programmation dynamique

In [20]:
def knapsack_dynamic(items, max_weight):
    n = len(items)

    V = [[0 for _ in range(max_weight + 1)] for _ in range(n + 1)]

    for k in range(1, n + 1):
        for y in range(1, max_weight + 1):
            if items[k-1].weight <= y:
                V[k][y] = max(V[k-1][y], items[k-1].value + V[k-1][y - items[k-1].weight])
            else:
                V[k][y] = V[k-1][y]

    return V

def find_optimal_items(items, V, max_weight):
    n = len(items)
    y = max_weight
    optimal_items = []

    for k in range(n, 0, -1):
        if V[k][y] != V[k-1][y]:
            optimal_items.append(items[k-1])
            y -= items[k-1].weight

    return optimal_items

items = [
    Item("lampe de poche", 68, 10),
    Item("boite d'allumettes", 50, 7),
    Item("boussole", 15, 6),
    Item("canif", 10, 5),
    Item("sac de couchage", 200, 20),
    Item("corde", 100, 15),
    Item("nourriture", 50, 17),
    Item("eau", 75, 18),
    Item("tente", 340, 25),
    Item("réchaud", 150, 24),
    Item("Pierre à feu", 75, 11),
]

max_weight = 250

n = len(items)

V = knapsack_dynamic(items, max_weight)
optimal_items = find_optimal_items(items, V, max_weight)

print("Valeur optimale:", V[len(items)][max_weight])
print("Objets à prendre:")
for item in optimal_items:
    print(f"- {item.name}")

print("\nLa complexité de l'algorithme est O(nW),")
print(f"où n = {n} (nombre d'objets) et W = {max_weight} (poids maximal).")



Valeur optimale: 61
Objets à prendre:
- eau
- nourriture
- corde
- canif
- boussole

La complexité de l'algorithme est O(nW),
où n = 11 (nombre d'objets) et W = 250 (poids maximal).


# Algorithme 4 : Calcul de la solution optimale exacte par la technique des couples dominants

add_dominated_pairs : Cette fonction ajoute de nouveaux couples en considérant le poids et la valeur de l'objet courant. Seuls les couples qui ne dépassent pas le poids maximal sont conservés.

remove_dominated_pairs : Cette fonction élimine les couples dominés. Elle trie d'abord les couples par poids, puis ne conserve que ceux qui ont une valeur non dominée.

knapsack_dominant_pairs : C'est la fonction principale qui utilise les deux fonctions ci-dessus pour construire la liste finale des couples et trouver la valeur optimale.

In [1]:
class Item:
    def __init__(self, name, weight, value):
        self.name = name
        self.weight = weight
        self.value = value

def add_dominated_pairs(couples, item, max_weight):
    new_couples = []
    for weight, value, selected_items in couples:
        new_weight = weight + item.weight
        new_value = value + item.value
        if new_weight <= max_weight:
            new_couples.append((new_weight, new_value, selected_items + [item.name]))
    return new_couples

def remove_dominated_pairs(couples):
    couples.sort()
    filtered = []
    max_value = -1
    for weight, value, selected_items in reversed(couples):
        if value > max_value:
            filtered.append((weight, value, selected_items))
            max_value = value
    return list(reversed(filtered))

def knapsack_dominant_pairs(items, max_weight):
    L = [(0, 0, [])]  
    for item in items:
        N = add_dominated_pairs(L, item, max_weight)
        L += N
        L = remove_dominated_pairs(L)
        print(f"Étape {items.index(item) + 1}: {[(w, p) for w, p, _ in L]}") 
    optimal_couple = max(L, key=lambda x: x[1])
    optimal_value = optimal_couple[1]
    optimal_items = optimal_couple[2] 
    return optimal_value, optimal_items

items = [
    Item("lampe de poche", 68, 10),
    Item("boite d'allumettes", 50, 7),
    Item("boussole", 15, 6),
    Item("canif", 10, 5),
    Item("sac de couchage", 200, 20),
    Item("corde", 100, 15),
    Item("nourriture", 50, 17),
    Item("eau", 75, 18),
    Item("tente", 340, 25),
    Item("réchaud", 150, 24),
    Item("Pierre à feu", 75, 11),
]

max_weight = 250
optimal_value, optimal_items = knapsack_dominant_pairs(items, max_weight)

print("\nValeur optimale:", optimal_value)
print("Objets à prendre:")
for item in optimal_items:
    print(f"- {item}")


Étape 1: [(68, 10)]
Étape 2: [(118, 17)]
Étape 3: [(133, 23)]
Étape 4: [(143, 28)]
Étape 5: [(143, 28)]
Étape 6: [(243, 43)]
Étape 7: [(243, 43)]
Étape 8: [(243, 43)]
Étape 9: [(243, 43)]
Étape 10: [(243, 43)]
Étape 11: [(243, 43)]

Valeur optimale: 43
Objets à prendre:
- lampe de poche
- boite d'allumettes
- boussole
- canif
- corde
